In [0]:
#cleaning data for spatial clustering

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
spark = SparkSession.builder.getOrCreate()



In [0]:
df1 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/tables/mv-collisions/Crashes.csv")
df2 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/tables/mv-collisions/Vehicles.csv")
df3 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/tables/mv-collisions/Persons.csv")

In [0]:
df1.count()

In [0]:
df2.count()

In [0]:
df1.printSchema()

In [0]:
df2.printSchema()

In [0]:
df3.printSchema()

In [0]:
df1.select("COLLISION_ID","LOCATION","ON STREET NAME","OFF STREET NAME","CROSS STREET NAME" , "BOROUGH" ).groupby("LOCATION","ON STREET NAME","OFF STREET NAME").count().show(100, truncate = False)

In [0]:
df1.where(F.col("LOCATION").isNull() & F.col("LATITUDE").isNull() & F.col("LONGITUDE").isNull() & (F.col("ON STREET NAME").isNotNull() | F.col("CROSS STREET NAME").isNotNull() | F.col("OFF STREET NAME").isNotNull())).count()
# .groupby("ON STREET NAME","OFF STREET NAME").count().show(50)
# .select("ON STREET NAME","OFF STREET NAME")

In [0]:
# df1.select("LOCATION","ON STREET NAME", "CROSS STREET NAME", "OFF STREET NAME").where((F.col("ON STREET NAME")=="20 AVENUE") & (F.col("CROSS STREET NAME") == "WHITESTONE EXPRESSWAY")).groupby("LOCATION","ON STREET NAME", "CROSS STREET NAME", "OFF STREET NAME").count().orderBy("count", ascending = False).show(50, truncate  = False)

df1.select("LOCATION","ON STREET NAME", "CROSS STREET NAME", "OFF STREET NAME").where((F.col("ON STREET NAME")=="20 AVENUE")).groupby("LOCATION","ON STREET NAME", "CROSS STREET NAME", "OFF STREET NAME").count().orderBy("count", ascending = False).show(50, truncate  = False)

In [0]:
df1.select("COLLISION_ID","LOCATION","ON STREET NAME","OFF STREET NAME","CROSS STREET NAME" , "BOROUGH").where(F.col("LOCATION").isNull() & (F.col("LATITUDE").isNotNull() & F.col("LONGITUDE").isNotNull())).show()

In [0]:
df1.where(F.col("LOCATION").isNull()).count()

In [0]:
df1.where(F.col("LATITUDE").isNull() & F.col("LONGITUDE").isNull()).count()

In [0]:
df1.select("COLLISION_ID","LOCATION","ON STREET NAME","OFF STREET NAME","CROSS STREET NAME" , "BOROUGH").where(F.col("LOCATION").isNull() & (F.col("ON STREET NAME").isNotNull())).show()

In [0]:
df1.select("LOCATION", "ON STREET NAME").where(F.col("LOCATION").isNull() & F.col("ON STREET NAME").isNotNull()).w

In [0]:
from pyspark.sql import Window
from pyspark.sql import functions as F


w = Window.partitionBy('ON STREET NAME', 'CROSS STREET NAME')

grouped = df1.filter(F.col("LOCATION").isNotNull()) \
    .withColumn('count', F.count("*").over(w)) \
    .withColumn('rn', F.row_number().over(w.orderBy(F.desc("count")))) \
    .where("rn = 1") \
    .select('LOCATION', 'ON STREET NAME', 'CROSS STREET NAME')

df1.alias("loc").join(grouped.alias("mode"), ['ON STREET NAME', 'CROSS STREET NAME'], "left") \
    .select("ON STREET NAME", "CROSS STREET NAME", F.coalesce("loc.LOCATION", "mode.LOCATION").alias("LOCATION_IMPUTED")) \
    .show()


In [0]:
df_imputed_loc = df1.alias("loc").join(grouped.alias("mode"), ['ON STREET NAME', 'CROSS STREET NAME'], "left") \
    .select("ON STREET NAME", "CROSS STREET NAME", F.coalesce("loc.LOCATION", "mode.LOCATION").alias("LOCATION_IMPUTED")) 


In [0]:
df_imputed_loc.where(F.col("LOCATION_IMPUTED").isNull() & F.col("OFF STREET NAME").isNotNull()).count()

In [0]:
df1.where(F.col("LOCATION").isNull()).count()

In [0]:
df1.where(F.col("LOCATION").isNull()).count()

In [0]:
df1.where(F.col("LOCATION").isNull() &(F.col("ON STREET NAME").isNull() & F.col("OFF STREET NAME").isNull() &F.col("CROSS STREET NAME").isNull())).count()

In [0]:
data = df1.filter(F.col("LOCATION").isNotNull() |(F.col("ON STREET NAME").isNotNull() | F.col("OFF STREET NAME").isNotNull() |F.col("CROSS STREET NAME").isNotNull()))

In [0]:
data.count()

In [0]:
data.where(F.col("LOCATION").isNull()).count()

In [0]:
data.where(F.col("LOCATION").isNull() & F.col("ON STREET NAME").isNotNull()).count()

In [0]:
data.where(F.col("LOCATION").isNull() & (F.col("ON STREET NAME").isNotNull())& (F.col("CROSS STREET NAME").isNotNull())).count()

In [0]:
data.where(F.col("LOCATION").isNull() & (F.col("ON STREET NAME").isNotNull())).count()

In [0]:
data.where(F.col("LOCATION").isNull() & (F.col("CROSS STREET NAME").isNotNull())).count()

In [0]:
data.where(F.col("LOCATION").isNull() & (F.col("OFF STREET NAME").isNotNull())).count()

In [0]:
coll = df1

In [0]:
def normalize_case(street_name):
    if street_name:
        return street_name.lower()# Normalize to sentence case
    else:
        return None

# UDF to apply the normalization function
normalize_case_udf = F.udf(normalize_case)

# Normalize casing of street names in DataFrame
df_1 = df1.withColumn("ON STREET NAME", normalize_case_udf("ON STREET NAME"))
df_1 = df1.withColumn("OFF STREET NAME", normalize_case_udf("OFF STREET NAME"))
df_1 = df_1.withColumn("CROSS STREET NAME", normalize_case_udf("CROSS STREET NAME"))

# Filter out rows where either LOCATION or any of the street names are not null
data = df_1.select("COLLISION_ID", "LOCATION", "ON STREET NAME", "OFF STREET NAME", "CROSS STREET NAME", "ZIP CODE", "BOROUGH").where(F.col("LOCATION").isNotNull() | 
                  (F.col("ON STREET NAME").isNotNull() | F.col("CROSS STREET NAME").isNotNull()))
from pyspark.sql.functions import when, col, mode, lit
from pyspark.sql.window import Window
data1 = data.withColumn("LOCATION", 
                        when(col("LOCATION").isNull(), 
                             when(col("ON STREET NAME").isNotNull() & col("CROSS STREET NAME").isNotNull(), 
                                  mode("LOCATION").over(Window.partitionBy("ON STREET NAME", "CROSS STREET NAME")))
                        ).otherwise(col("LOCATION")))

data2 = data1.withColumn("LOCATION", 
                        when(col("LOCATION").isNull(), 
                             when(col("ON STREET NAME").isNotNull() & col("CROSS STREET NAME").isNull(), 
                                  mode("LOCATION").over(Window.partitionBy("ON STREET NAME")))
                        ).otherwise(col("LOCATION")))       
data2.where(col('LOCATION').isNull()).count() 
data3 = data2.withColumn("LOCATION", 
                        when(col("LOCATION").isNull(), 
                             when(col("ON STREET NAME").isNull() & col("CROSS STREET NAME").isNotNull(), 
                                  mode("LOCATION").over(Window.partitionBy("CROSS STREET NAME")))
                             
                        ).otherwise(col("LOCATION")))
# data4 = data3.withColumn("LOCATION", 
#                         when(col("LOCATION").isNull(), 
#                              when(col("ON STREET NAME").isNull() & col("CROSS STREET NAME").isNotNull() & col("OFF STREET NAME").isNull(), 
#                                   mode("LOCATION").over(Window.partitionBy("CROSS STREET NAME")))
                             
#                         ).otherwise(col("LOCATION")))
data3.where(col('LOCATION').isNull()).count()

In [0]:
data3.select("ON STREET NAME", "CROSS STREET NAME").where(F.col("LOCATION").isNull()).groupby("ON STREET NAME", "CROSS STREET NAME").count().orderBy("count", ascending = False).show(50, truncate = False)

In [0]:
df1.select("ON STREET NAME" ,"CROSS STREET NAME","LOCATION").where((F.col("LOCATION").isNotNull())& (F.col("ON STREET NAME") == "UTICA AVENUE") & (F.col("CROSS STREET NAME") == "EAST NEW YORK AVENUE")).show()

In [0]:
coll = df1

In [0]:
coll = coll.dropna(how="all", subset=["LOCATION", "ON STREET NAME", "CROSS STREET NAME"])

# Show the DataFrame after dropping rows
coll.count()

In [0]:
coll1 = coll.withColumn("LOCATION", 
                        when(col("LOCATION").isNull(), 
                             when(col("ON STREET NAME").isNotNull() & col("CROSS STREET NAME").isNotNull(), 
                                  mode("LOCATION").over(Window.partitionBy("ON STREET NAME", "CROSS STREET NAME")))
                             .when(col("ON STREET NAME").isNotNull() & col("CROSS STREET NAME").isNull(), 
                                  mode("LOCATION").over(Window.partitionBy("ON STREET NAME")))
                             .when(col("ON STREET NAME").isNull() & col("CROSS STREET NAME").isNotNull(), 
                                  mode("LOCATION").over(Window.partitionBy("CROSS STREET NAME")))
                             .when(col("ON STREET NAME").isNotNull(), 
                                  mode("LOCATION").over(Window.partitionBy("ON STREET NAME")))
                             .when(col("CROSS STREET NAME").isNotNull(), 
                                  mode("LOCATION").over(Window.partitionBy("CROSS STREET NAME")))
                        ).otherwise(col("LOCATION")))

In [0]:
coll1.where(col('LOCATION').isNull()).count()

In [0]:
coll2 = coll1.select("ON STREET NAME", "CROSS STREET NAME").where(F.col("LOCATION").isNull()).groupby("ON STREET NAME", "CROSS STREET NAME").count().orderBy("count", ascending = False)

In [0]:
coll2.count()

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
df1 = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/tables/mv-collisions/Crashes.csv")
coll = df1

def normalize_case(street_name):
    if street_name:
        return street_name.lower()# Normalize to sentence case
    else:
        return None

# UDF to apply the normalization function
normalize_case_udf = F.udf(normalize_case)

# Normalize casing of street names in DataFrame
coll = coll.withColumn("ON STREET NAME", normalize_case_udf("ON STREET NAME"))
coll = coll.withColumn("CROSS STREET NAME", normalize_case_udf("CROSS STREET NAME"))


# Define a window partitioned by 'ON STREET NAME' and 'CROSS STREET NAME'
window = Window.partitionBy('ON STREET NAME', 'CROSS STREET NAME')

# Define a window partitioned by 'ON STREET NAME'
window_on_street = Window.partitionBy('ON STREET NAME')

# Define a window partitioned by 'CROSS STREET NAME'
window_cross_street = Window.partitionBy('CROSS STREET NAME')

# Create a new column 'LOCATION_TEMP' which takes the mode of 'LOCATION' over the defined window
coll = coll.withColumn('LOCATION_TEMP', F.mode("LOCATION").over(window))

# If 'LOCATION_TEMP' is still null, then take the mode of 'LOCATION' over the 'ON STREET NAME' window
coll = coll.withColumn('LOCATION_TEMP', F.when(F.col('LOCATION_TEMP').isNull(), F.mode("LOCATION").over(window_on_street)).otherwise(F.col('LOCATION_TEMP')))

# If 'LOCATION_TEMP' is still null, then take the mode of 'LOCATION' over the 'CROSS STREET NAME' window
coll = coll.withColumn('LOCATION_TEMP', F.when(F.col('LOCATION_TEMP').isNull(), F.mode("LOCATION").over(window_cross_street)).otherwise(F.col('LOCATION_TEMP')))

# Replace null values in 'LOCATION' with the values from 'LOCATION_TEMP'
coll = coll.withColumn('LOCATION', F.when(F.col('LOCATION').isNull(), F.col('LOCATION_TEMP')).otherwise(F.col('LOCATION')))

# Drop the temporary column 'LOCATION_TEMP'
coll = coll.drop('LOCATION_TEMP')
coll = coll.dropna(how="all", subset=["LOCATION"])
coll.count()

In [0]:
coll.where(col('LOCATION').isNull()).count()

In [0]:
coll2 = coll.select("ON STREET NAME", "CROSS STREET NAME").where(F.col("LOCATION").isNull()).groupby("ON STREET NAME", "CROSS STREET NAME").count().orderBy("count", ascending = False)

In [0]:
coll2.count()

In [0]:
coll2.show(10, truncate=False)

In [0]:
df1.select("ON STREET NAME" ,"CROSS STREET NAME","LOCATION").where((F.col("LOCATION").isNotNull())& (F.col("ON STREET NAME") == "BELT PARKWAY") & (F.col("CROSS STREET NAME") == "150 STREET")).show()

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
data = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/tables/mv-collisions/Crashes.csv")
collisions = data

def normalize_case(street_name):
    if street_name:
        return street_name.lower()# Normalize to sentence case
    else:
        return None

# UDF to apply the normalization function
normalize_case_udf = F.udf(normalize_case)

# Normalize casing of street names in DataFrame
collisions = collisions.withColumn("ON STREET NAME", normalize_case_udf("ON STREET NAME"))
collisions = collisions.withColumn("CROSS STREET NAME", normalize_case_udf("CROSS STREET NAME"))


# Define a window partitioned by 'ON STREET NAME' and 'CROSS STREET NAME'
window = Window.partitionBy('ON STREET NAME', 'CROSS STREET NAME')

# Define a window partitioned by 'ON STREET NAME'
window_on_street = Window.partitionBy('ON STREET NAME')

# Define a window partitioned by 'CROSS STREET NAME'
window_cross_street = Window.partitionBy('CROSS STREET NAME')

# Create a new column 'LOCATION_TEMP' which takes the mode of 'LOCATION' over the defined window
collisions = collisions.withColumn('LOCATION_TEMP', F.mode("LOCATION").over(window))

# If 'LOCATION_TEMP' is still null, then take the mode of 'LOCATION' over the 'ON STREET NAME' window
collisions = collisions.withColumn('LOCATION_TEMP', F.when(F.col('LOCATION_TEMP').isNull(), F.mode("LOCATION").over(window_on_street)).otherwise(F.col('LOCATION_TEMP')))

# If 'LOCATION_TEMP' is still null, then take the mode of 'LOCATION' over the 'CROSS STREET NAME' window
collisions = collisions.withColumn('LOCATION_TEMP', F.when(F.col('LOCATION_TEMP').isNull(), F.mode("LOCATION").over(window_cross_street)).otherwise(F.col('LOCATION_TEMP')))

# Replace null values in 'LOCATION' with the values from 'LOCATION_TEMP'
collisions = collisions.withColumn('LOCATION', F.when(F.col('LOCATION').isNull(), F.col('LOCATION_TEMP')).otherwise(F.col('LOCATION')))

# Drop the temporary column 'LOCATION_TEMP'
collisions = collisions.drop('LOCATION_TEMP')


In [0]:
collisions.count()

In [0]:
collisions = collisions.dropna(how="all", subset=["LOCATION"])
collisions.count()

In [0]:
data.count()

In [0]:
collisions.where(F.col("LOCATION").isNull()).count()

In [0]:
collisions.printSchema()

In [0]:

csv_path = "dbfs:/FileStore/tables/mv-collisions/collisions_clean.csv"
collisions.write.format("csv").option("header", "true").save(csv_path)


In [0]:
data = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/tables/mv-collisions/collisions_clean.csv")

In [0]:
data.show(10)

In [0]:
collisions.select("LOCATION").where(F.col("LOCATION")== "0").show(truncate = False)